# PHYS 3116 SAMI

In [ ]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# read in all data files.
InputCatGAMADR3 = pd.read_csv('InputCatGAMADR3.csv')
samiDR3InputCatClusters = pd.read_csv('samiDR3InputCatClusters.csv')
samiDR3StelKin = pd.read_csv('samiDR3StelKin.csv')
samiDR3VisualMorphology = pd.read_csv('samiDR3VisualMorphology.csv')

In [4]:
# small check
InputCatGAMADR3.head(5)

,Unnamed: 0,catid,ra_obj,dec_obj,ra_ifu,dec_ifu,r_petro,r_auto,z_tonry,z_spec,...,mu_within_1re,mu_1re,mu_2re,ellip,pa,mstar,g_i,a_g,surv_sami,bad_class
0,0,40873,183.439765,-0.833229,183.439765,-0.833229,19.491,16.869,0.05234,0.05110,...,NaN,NaN,NaN,NaN,NaN,9.81,0.76,0.082,3,3
1,1,48805,218.621562,-0.798935,218.621562,-0.798935,19.507,17.607,0.03735,0.03654,...,NaN,NaN,NaN,NaN,NaN,8.92,0.39,0.149,3,3
2,2,196977,133.590423,-0.707572,133.590423,-0.707572,17.149,17.130,0.05055,0.04949,...,NaN,NaN,NaN,NaN,NaN,9.96,1.13,0.092,3,0
3,3,215293,133.536926,0.498455,133.536926,0.498455,19.755,15.062,0.02944,0.02842,...,NaN,NaN,NaN,NaN,NaN,10.10,0.84,0.184,8,3
4,4,323508,132.033380,1.564864,132.033380,1.564864,19.154,18.930,0.04058,0.03956,...,NaN,NaN,NaN,NaN,NaN,8.70,0.63,0.134,3,3
